# 【Face Alignment其二】face alignment的3D投影算法

本文参考代码为：https://github.com/dougsouza/face-frontalization

在《【Face Alignment其一】用dlib进行face alignment的简单实现》中，讲解了Face Alignment的简单实现。但通常这种程度的alignment是不够的。目前一个主流的做法是利用人脸landmark的3D模型，将facial landmark投影至3D空间后再投影到正面上。